# Assignment 10

## 1.Edit Distance

In [1]:
solution = {}

In [2]:
from functools import lru_cache

In [3]:
#把s1的值变成和s2值一样
@lru_cache(maxsize=2**50)
def edit_distance(string1, string2):

    #修复了某个字符串为0后，没有添加解决方案直接返回的bug
    if len(string1) == 0:#添加string1为0时的解决方案，添加string1
        str2 = string2
        while(True):
            if len(str2) == 0:
                solution[(string1, str2)] = ''
                break
            if (string1, str2) not in solution:
                solution[(string1, str2)] = 'ADD {}'.format(str2[-1])
            str2 = str2[:-1]
        return len(string2)#删除s2的字母
    if len(string2) == 0:#添加string2为0时的解决方案，删除string1
        str1 = string1
        while(True):
            if (len(str1) == 0):
                solution[(str1, string2)] = ''
                break
            if (str1, string2) not in solution:
                solution[(str1, string2)] = 'DEL {}'.format(str1[-1])
            str1 = str1[:-1]
        return len(string1)#添加s1的字母
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  
        # string 1 delete tail
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  
        # string 1 add tail of string2
    ]
    
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

    candidates.append(both_forward)
    
    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    solution[(string1, string2)] = operation 
    
    return min_distance

In [4]:
edit_distance('ABCDECG','ABCCEF')

3

In [5]:
solution

{('', 'ABCCEF'): 'ADD F',
 ('', 'ABCCE'): 'ADD E',
 ('', 'ABCC'): 'ADD C',
 ('', 'ABC'): 'ADD C',
 ('', 'AB'): 'ADD B',
 ('', 'A'): 'ADD A',
 ('', ''): '',
 ('A', ''): 'DEL A',
 ('A', 'A'): '',
 ('A', 'AB'): 'ADD B',
 ('A', 'ABC'): 'ADD C',
 ('A', 'ABCC'): 'ADD C',
 ('A', 'ABCCE'): 'ADD E',
 ('A', 'ABCCEF'): 'ADD F',
 ('AB', ''): 'DEL B',
 ('AB', 'A'): 'DEL B',
 ('AB', 'AB'): '',
 ('AB', 'ABC'): 'ADD C',
 ('AB', 'ABCC'): 'ADD C',
 ('AB', 'ABCCE'): 'ADD E',
 ('AB', 'ABCCEF'): 'ADD F',
 ('ABC', ''): 'DEL C',
 ('ABC', 'A'): 'DEL C',
 ('ABC', 'AB'): 'DEL C',
 ('ABC', 'ABC'): '',
 ('ABC', 'ABCC'): 'ADD C',
 ('ABC', 'ABCCE'): 'ADD E',
 ('ABC', 'ABCCEF'): 'ADD F',
 ('ABCD', ''): 'DEL D',
 ('ABCD', 'A'): 'DEL D',
 ('ABCD', 'AB'): 'DEL D',
 ('ABCD', 'ABC'): 'DEL D',
 ('ABCD', 'ABCC'): 'SUB D => C',
 ('ABCD', 'ABCCE'): 'ADD E',
 ('ABCD', 'ABCCEF'): 'ADD F',
 ('ABCDE', ''): 'DEL E',
 ('ABCDE', 'A'): 'DEL E',
 ('ABCDE', 'AB'): 'DEL E',
 ('ABCDE', 'ABC'): 'DEL E',
 ('ABCDE', 'ABCC'): 'DEL E',
 ('AB

In [6]:
#解决方案方法
def parse_solution(str1,str2):
    #print('str1:{}'.format(str1))
    #print('str2:{}'.format(str2))
    operation =  solution[(str1, str2)]
    if str1 == str2 and len(operation) == 0:
        return [operation]

    if len(operation) == 0 or operation.find('SUB') >= 0:
        str1 = str1[:-1]
        str2 = str2[:-1]

    elif operation.find('DEL') >= 0:
        str1 = str1[:-1]

    elif operation.find('ADD') >= 0:
        str2 = str2[:-1]
    
    return [operation] + parse_solution(str1,str2)

In [7]:
print(parse_solution('ABCDECG', 'ABCCEF'))

['DEL G', 'SUB C => F', '', 'SUB D => C', '']


In [8]:
#合并方法
str1 = 'B'
str2 = 'ABCCEF'
edit_distance(str1,str2)
solutions = parse_solution(str1, str2)
print(solutions)

['ADD F', 'ADD E', 'ADD C', 'ADD C', '', 'ADD A', '']


In [9]:
#合并方法
str1 = ''
str2 = 'ABCCEF'
edit_distance(str1,str2)
solutions = parse_solution(str1, str2)
print(solutions)

['ADD F', 'ADD E', 'ADD C', 'ADD C', 'ADD B', 'ADD A', '']


## 2.Pinyin Auto Correction Problem

In [1]:
chinese_dataset = 'D:/Repos/MyPython/Nlp/assignment_1/article_9k.txt'
CHINESE_CHARATERS = open(chinese_dataset).read()
CHINESE_CHARATERS[:40]

'此外自本周6月12日起除小米手机6等15款机型外其余机型已暂停更新发布含开发版体'

In [7]:
import pinyin
def chinese_to_pinyin(character):
    return pinyin.get(character, format='strip', delimiter=' ')

In [13]:
CHINESE_CHARATERS_COPYS = chinese_to_pinyin(CHINESE_CHARATERS)

In [5]:
len(CHINESE_CHARATERS_COPYS)

129433034

In [10]:
import re
def tokens(text):
    "List all the pinyin characters"
    return re.findall('[a-z]+',text.lower())

In [8]:
CHINESE_CHARATERS_COPYS[:100]

'ci wai zi ben zhou 6 yue 1 2 ri qi chu xiao mi shou ji 6 deng 1 5 kuan ji xing wai qi yu ji xing yi '

In [9]:
tokens(CHINESE_CHARATERS_COPYS[:35])[:5]

['ci', 'wai', 'zi', 'ben', 'zhou']

In [10]:
pinyin_token = tokens(CHINESE_CHARATERS_COPYS)

In [11]:
#写入文件
with open('article_9k_pinyin.txt', 'w') as f:
    for a in pinyin_token:
        f.write(a + '\n')

In [8]:
from collections import Counter, defaultdict

#### 从文件读取拼音并count、或从内存count

In [11]:
#读取文件并count
pinyin_token = tokens(open('article_9k_pinyin.txt').read())
PINYIN_COUNT = Counter(pinyin_token)

In [14]:
#读取内存并count
PINYIN_COUNT = Counter(tokens(CHINESE_CHARATERS_COPYS))

In [15]:
PINYIN_COUNT.most_common()[:10]

[('shi', 860634),
 ('de', 809887),
 ('n', 691114),
 ('yi', 682478),
 ('ji', 645276),
 ('guo', 430042),
 ('zhong', 409418),
 ('zhi', 398612),
 ('xin', 359619),
 ('li', 355441)]

In [51]:
#测试count
PINYIN_COUNT.get('ci')

95927

In [16]:
def correct(word):
    'Find the most possible pinyin based on edit distance'
    # Prefer edit distance 0, then 1, then 2; otherwist default to word itself
    candidates = (known(edits0(word)) or
                  known(edits1(word)) or
                  known(edits2(word)) or
                  [word])
    return max(candidates,key=PINYIN_COUNT.get)

In [17]:
def known(words):
    'Return the pinyin in our data'
    return {w for w in words if w in PINYIN_COUNT}

def edits0(word):
    'Return all strings that are zero edits away from word (i.e., just word itself).'
    return {word}

def edits2(word):
    'Return all strings that are two edits away from this pinyin.'
    return {e2 
        for e1 in edits1(word) 
            for e2 in edits1(e1)}

In [18]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'

def splits(word):
    'Return a list of all possible (first, rest) pairs that comprise pinyin.'
    return [(word[:i], word[i:])
           for i in range(len(word)+1)]

def edits1(word):
    'Return all strings that are one edit away from this pinyin.'
    pairs = splits(word)

    #删除，在中间删除一个字母
    deletes = [a+b[1:] for (a,b) in pairs if b]

    #交换顺序，将两个字母交换顺序
    transposes = [a+b[1]+b[0]+b[2:] for (a,b) in pairs if len(b) > 1]

    #替换，在中间替换一个字母
    replaces = [a+c+b[1:] for (a,b) in pairs for c in alphabet if b]

    #插入，在中间插入一个字母
    inserts = [a+c+b for (a,b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

In [18]:
splits('pinyin')

[('', 'pinyin'),
 ('p', 'inyin'),
 ('pi', 'nyin'),
 ('pin', 'yin'),
 ('piny', 'in'),
 ('pinyi', 'n'),
 ('pinyin', '')]

In [38]:
print(edits0('pinyin'))

{'pinyin'}


In [39]:
print(edits1('pinyin'))

{'pinyifn', 'pxinyin', 'pniyin', 'pinyinr', 'tinyin', 'pinyinp', 'finyin', 'pinyib', 'pinyik', 'ppinyin', 'plnyin', 'piuyin', 'pinyink', 'pidyin', 'pcinyin', 'pqinyin', 'piniyin', 'pinsin', 'pinmin', 'pindin', 'pznyin', 'pinyiv', 'upinyin', 'pinyil', 'pimnyin', 'wpinyin', 'phnyin', 'pinyzin', 'pifnyin', 'pikyin', 'pinygn', 'pinain', 'pinyihn', 'pijyin', 'pinmyin', 'pizyin', 'pinyfin', 'ppnyin', 'pinycin', 'pinuin', 'pinyig', 'fpinyin', 'pinyinn', 'pdinyin', 'pinyina', 'pinyih', 'bpinyin', 'pinlin', 'uinyin', 'pinvin', 'pinyiin', 'pinyon', 'pjnyin', 'pwnyin', 'ainyin', 'pninyin', 'psnyin', 'pibnyin', 'pinin', 'pinyyn', 'pinyins', 'pinytn', 'jinyin', 'pinylin', 'pinein', 'pinlyin', 'pinkyin', 'pinybin', 'pxnyin', 'vpinyin', 'pigyin', 'pinyinc', 'minyin', 'pinayin', 'pminyin', 'pinhin', 'qinyin', 'piiyin', 'pbinyin', 'pinydin', 'pinyvin', 'punyin', 'pingin', 'pinsyin', 'pinhyin', 'pinfin', 'pinrin', 'pinyoin', 'pinywn', 'pinyln', 'pinyiny', 'kinyin', 'pinying', 'pinyio', 'piayin', 'rinyin

In [40]:
correct('yin')

'yin'

In [19]:
def correct_sequence_pinyin(text_pinyin):
    #map() 会根据提供的函数对指定序列做映射
    #即对集合中的每个元素执行map的方法
    return ' '.join(map(correct, text_pinyin.split()))

In [43]:
correct_sequence_pinyin('zhe sih yi ge ce sho')

'zhe shi yi ge ce shi'

In [44]:
correct_sequence_pinyin('wo xiang shagn qinng hua da xue')

'wo xiang shang qing hua da xue'

### 如何在不带空格的时候完成自动修整？--> 如何完成拼音的自动分割？ 

woyaoshangqinghua   
w yaoshangqinghua   
wo yaoshangqinghua   
woyao shangqinghua   

-> DP

In [21]:
#大致思想，切分成两段，分别使用2gram计算left和right
splits('woyaoshangqinghua')

[('', 'woyaoshangqinghua'),
 ('w', 'oyaoshangqinghua'),
 ('wo', 'yaoshangqinghua'),
 ('woy', 'aoshangqinghua'),
 ('woya', 'oshangqinghua'),
 ('woyao', 'shangqinghua'),
 ('woyaos', 'hangqinghua'),
 ('woyaosh', 'angqinghua'),
 ('woyaosha', 'ngqinghua'),
 ('woyaoshan', 'gqinghua'),
 ('woyaoshang', 'qinghua'),
 ('woyaoshangq', 'inghua'),
 ('woyaoshangqi', 'nghua'),
 ('woyaoshangqin', 'ghua'),
 ('woyaoshangqing', 'hua'),
 ('woyaoshangqingh', 'ua'),
 ('woyaoshangqinghu', 'a'),
 ('woyaoshangqinghua', '')]

In [22]:
def mycountget(pinyin):
    ret = PINYIN_COUNT.get(pinyin)
    return 0 if ret is None else ret

In [23]:
splittest = splits('woyaoshangqinghua')
test = [a for (a,b) in splittest if a]
max(test,key=mycountget)

'wo'

In [24]:
splittest = splits('yaoshangqinghua')
test = [a for (a,b) in splittest if a]
max(test,key=mycountget)

'yao'

### 处理拼音的2gram文件

In [ ]:
#从原始文件中读取，并处理成不带任何数字、英文、特殊符号的2gram
import pandas as pd
content = pd.read_csv('D:/Repos/MyPython/Nlp/sqlResult_1558435.csv', encoding='gb18030')

content[u'content'] = content[u'content'].astype(str)
content[u'content'] = content[u'content'].apply(lambda x : x.replace('\r',''))
content[u'content'] = content[u'content'].apply(lambda x : x.replace('\n',''))
content[u'content'] = content[u'content'].apply(lambda x : x.replace('\u3000',''))
content[u'content'] = content[u'content'].apply(lambda x : x.replace('\\n',''))
content[u'content'] = content[u'content'].apply(lambda x : x.replace(' ',''))
articles = content['content'].tolist()

In [26]:
''.join(re.findall('[\u4e00-\u9fa5]+',articles[11580]))

'俄美乌总统在诺曼底谈啥惠晓霜俄罗斯总统弗拉基米尔普京日在法国诺曼底与美国总统贝拉克奥巴马交谈这是乌克兰危机爆发俄美关系恶化以来两人首次面对面同一天普京还与定于日就职的乌克兰当选总统彼得罗波罗申科首次会面小标题波罗申科好的开始波罗申科与普京握手交谈大约分钟讨论乌克兰局势和两国关系谈得不轻松波罗申科在会面结束后说当我提及克里米亚问题对方反应不悦不过对话已经开始是好事他说波罗申科在月日举行的乌克兰总统选举中获胜定于月日宣誓就职乌克兰时任总统维克托亚努科维奇月在反对派示威中被议会罢免职务并出走俄罗斯后俄方指责乌克兰反对派政变法国总统弗朗索瓦奥朗德说普京与波罗申科讨论俄罗斯如何承认乌克兰选举以及化解乌克兰紧张局势的措施交谈时间不长但是足够传达信息奥朗德告诉法国媒体路透社以一名不愿公开身份的法国高级官员为消息源报道两人还讨论了波罗申科就职演说的关键内容如果一切顺利他们周一日将再次交谈以保持接触波罗申科说一名俄方代表将抵达乌克兰双方日开始讨论化解危机的初期步骤波罗申科预计化解危机方案落实的可能性大所有问题都很棘手他返回乌克兰前告诉美联社记者但是我们将全力以赴以达成我们自己设定的目标将于周日开始谈判小标题普京先展诚意再谈关系克里姆林宫方面说普京与波罗申科在交谈中都同意应尽快结束乌克兰东部武装冲突乌克兰政府军和乌克兰联邦制支持者都应停止敌对行动以政治途径和平解决是唯一出路普京告诉俄罗斯媒体他欢迎波罗申科的立场即必须立即结束乌东部流血冲突他有一套计划他快速向我做了解释我不能确定计划在实际中如何落实但是总体而言它的方向正确普京要求乌方展示诚意停止镇压行动从而可为我们在其他领域的关系包括经济关系创造条件俄乌关系恶化后俄方取消了对乌供应天然气的优惠价格催促乌方偿还所欠气费乌方则指责俄方经济侵略普京说他与波罗申科交谈期间没有讨论天然气供应价格但是我知道俄罗斯天然气工业公司与他们的乌克兰伙伴已经接近达成一项明确的合同如果他们乌方偿付欠款我方不排除向乌方展现姿态给予某些支持普京说小标题奥巴马改善美俄关系有条件普京在诺曼底还与美国总统奥巴马举行非正式会面克里姆林宫说双方讨论乌克兰局势以及乌东部危机都赞成尽快结束暴力和军事行动这是乌克兰危机爆发后俄美总统首次面对面交谈普京告诉俄罗斯媒体交换意见是非常积极的事与奥巴马的交谈颇具实质性美国总统国家安全事务副助理本杰明罗兹说奥巴马在交谈中就消解危机向俄方提

In [25]:
import jieba
def cut(string): return list(jieba.cut(string))

In [29]:
#处理成不带任何数字、英文、特殊符号的token，避免特殊情况影响2gram
token = []
for i in range(len(articles)):
    token += cut(''.join(re.findall('[\u4e00-\u9fa5]+',articles[i])))
    if i % 5000 == 0:
        print('{} finished'.format(i))

0 finished
5000 finished
10000 finished
15000 finished
20000 finished
25000 finished
30000 finished
35000 finished
40000 finished
45000 finished
50000 finished
55000 finished
60000 finished
65000 finished
70000 finished
75000 finished
80000 finished
85000 finished


In [30]:
#处理成2gram
token_2gram = [''.join(token[i:i+2]) for i in range(len(token[:-2]))]

In [32]:
token_2gram[:10]

['此外自', '自本周', '本周月', '月日', '日起', '起除', '除小米', '小米手机', '手机等款', '等款机型']

In [35]:
#保存2gram到文件
with open('article_9k_chinese_2gram.txt', 'w') as f:
    for a in token_2gram:
        f.write(a + '\n')

In [36]:
#读取2gram文件
token_2gram = []
for i, line in enumerate((open('article_9k_chinese_2gram.txt'))):
    token_2gram.append(''.join(line.replace('\n','')))

In [38]:
token_2gram[:10]

['此外自', '自本周', '本周月', '月日', '日起', '起除', '除小米', '小米手机', '手机等款', '等款机型']

In [39]:
#测试2gram拼音
tokens(chinese_to_pinyin(token_2gram[0]))

['ci', 'wai', 'zi']

In [40]:
#写入2gram拼音文件
i = 0
with open('article_9k_pinyin_2gram.txt', 'w') as f:
    for a in token_2gram:
        i += 1
        pinyin_2gram = tokens(chinese_to_pinyin(a))
        if len(pinyin_2gram) > 1 :
            f.write(' '.join(pinyin_2gram) + '\n')
        if i % 500000 == 0:
            print('{} finished'.format(i))

500000 finished
1000000 finished
1500000 finished
2000000 finished
2500000 finished
3000000 finished
3500000 finished
4000000 finished
4500000 finished
5000000 finished
5500000 finished
6000000 finished
6500000 finished
7000000 finished
7500000 finished
8000000 finished
8500000 finished
9000000 finished
9500000 finished
10000000 finished
10500000 finished
11000000 finished
11500000 finished
12000000 finished
12500000 finished
13000000 finished
13500000 finished
14000000 finished
14500000 finished
15000000 finished
15500000 finished


In [41]:
#从文件读取2gram拼音
pinyin_token_2gram = []
for i, line in enumerate((open('article_9k_pinyin_2gram.txt'))):
    pinyin_token_2gram.append(''.join(line.replace('\n','').replace(' ','')))

In [42]:
pinyin_token_2gram[:10]

['ciwaizi',
 'zibenzhou',
 'benzhouyue',
 'yueri',
 'riqi',
 'qichu',
 'chuxiaomi',
 'xiaomishouji',
 'shoujidengkuan',
 'dengkuanjixing']

In [43]:
PINYIN_2gram_COUNT = Counter(pinyin_token_2gram)

In [44]:
PINYIN_2gram_COUNT.most_common()[:10]

[('yueri', 75439),
 ('waidaierxian', 60469),
 ('xinhuashezhaopian', 50401),
 ('xinhuashejizhe', 30865),
 ('rizai', 25311),
 ('nianyueriwaidai', 20851),
 ('zhaopianwaidai', 20777),
 ('waidainianyueri', 20763),
 ('bisaizhong', 20574),
 ('shexinhuashe', 18414)]

In [46]:
PINYIN_2gram_COUNT.get('woyao')

124

In [47]:
#编写测试方法
def mycountget1(pinyin):
    ret = PINYIN_2gram_COUNT.get(pinyin)
    return 0 if ret is None else ret

In [48]:
#测试1
splittest = splits('woyaoshangqinghua')
test = [a for (a,b) in splittest if a]
max(test,key=mycountget1)

'woyao'

In [49]:
#测试2
splittest = splits('shangqinghua')
test = [a for (a,b) in splittest if a]
max(test,key=mycountget1)

'shangqi'

#### 处理逻辑开始

In [99]:
#修改原始的know方法使用PINYIN_2gram_COUNT
def known2(words):
    'Return the pinyin in our data'
    return {w for w in words if w in PINYIN_2gram_COUNT}

In [126]:
#修改原始的correct方法使用PINYIN_2gram_COUNT
from functools import lru_cache
@lru_cache(maxsize=2**10)
def correct2(word):
    'Find the most possible pinyin based on edit distance'
    # Prefer edit distance 0, then 1, then 2; otherwist default to word itself
    candidates = (known2(edits0(word)) or
                  known2(edits1(word)) or
                  known2(edits2(word)) or
                  [word])
    return max(candidates,key=PINYIN_2gram_COUNT.get)

In [121]:
#使用splits切分成左右，在分别计算左右的2gram
def correct_sequence_pinyin2(text_pinyin):
    ret = {}
    splitstr = splits(text_pinyin)
    for i in range(len(splitstr)):
        left = splitstr[i][0]
        right = splitstr[i][1]
        leftfix = correct2(left)
        leftcount = PINYIN_2gram_COUNT.get(leftfix)
        rightfix = correct2(right)
        rightcount = PINYIN_2gram_COUNT.get(rightfix)
        ret[leftfix + ' ' + rightfix] = leftcount if leftcount is not None else 0 + rightcount if rightcount is not None else 0
    return max(ret, key = lambda x:ret[x])#返回概率最大的

In [124]:
#查看woyao的概率
PINYIN_2gram_COUNT.get('woyao')

124

In [115]:
#查看woyaoshang的概率
PINYIN_2gram_COUNT.get('woyaoshang')

18

In [112]:
#查看nihua的概率
PINYIN_2gram_COUNT.get('nihua')

3

In [113]:
#查看qinghua的概率
PINYIN_2gram_COUNT.get('qinghua')

In [122]:
#纠错测试1
correct_sequence_pinyin2('woyaoshangqinghua')

'woye shangqinghua'

In [127]:
#纠错测试2
correct_sequence_pinyin2('woxiangshagnqinnghuadaxue')

'woxiang shangqinghuadaxue'

#### 总结:执行效率有点慢，但是correct2加了缓存后提升明显